In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM, Convolution1D, GlobalMaxPooling1D
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from matplotlib import pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

import seaborn as sns
# Word2vec
import gensim

# Utility
import ssl
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

In [2]:
df = pd.read_csv('d1.csv')
print(df.iloc[:10])

   Unnamed: 0                                            message  label
0           0  The lack of this understanding is a small but ...      1
1           1  i just told my parents about my depression and...      1
2           2  depression is something i don't speak about ev...      1
3           3  Made myself a tortilla filled with pb&j. My de...      1
4           4  @WorldofOutlaws I am gonna need depression med...      1
5           5  my anxiety and my depression fighting over who...      1
6           6  wow she's suddenly cured my depression and gav...      1
7           7  I am officially done with @kanyewest. him, the...      1
8           8  Me: what's wrong?My girl: *looks up at me with...      1
9           9  @AusBorderForce @PeterDutton_MP @shanebazzi Ag...      1


In [3]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/santhosh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [5]:
def preprocess(text, stem=True):
    # Remove link,user and special characters
    TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [6]:
print(df.head())
df.message = df.message.apply(lambda x: preprocess(x))

print("after preprocessing")
print(df.head())

   Unnamed: 0                                            message  label
0           0  The lack of this understanding is a small but ...      1
1           1  i just told my parents about my depression and...      1
2           2  depression is something i don't speak about ev...      1
3           3  Made myself a tortilla filled with pb&j. My de...      1
4           4  @WorldofOutlaws I am gonna need depression med...      1
after preprocessing
   Unnamed: 0                                            message  label
0           0  lack understand small signific part caus anxie...      1
1           1  told parent depress hard get gen x peopl under...      1
2           2  depress someth speak even go also doubl edg sw...      1
3           3  made tortilla fill pb j depress cure olivia 1 ...      1
4           4  gonna need depress med soon rainout spin equil...      1


In [7]:
texts = df.message.to_numpy()
labels = df.label.to_numpy()

In [11]:
# Initialize TF-IDF vectorizer with maximum features set to 1000
vectorizer = TfidfVectorizer(max_features=1000)

# Fit and transform the texts into TF-IDF features
tfidf_features = vectorizer.fit_transform(texts)

# Set the maximum sequence length to the number of features in the TF-IDF matrix
max_len = tfidf_features.shape[1]

# Split the dataset into train and test sets


In [9]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_features, labels, test_size=0.2, random_state=42)

In [12]:
tfidf_features.shape[1]

1000

In [13]:
# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=tfidf_features.shape[1], output_dim=128, input_length=max_len, mask_zero=True))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Train the LSTM model
model.fit(X_train.toarray(), y_train, epochs=8, batch_size=128, validation_data=(X_test.toarray(), y_test))

Epoch 1/8


2023-04-25 23:55:50.400272: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


51/51 [==============================] - 178s 3s/step - loss: 0.5976 - accuracy: 0.6433 - val_loss: 0.5616 - val_accuracy: 0.6652
Epoch 2/8
51/51 [==============================] - 412s 8s/step - loss: 0.5738 - accuracy: 0.6740 - val_loss: 0.5770 - val_accuracy: 0.6450
Epoch 3/8
51/51 [==============================] - 535s 11s/step - loss: 0.5709 - accuracy: 0.6743 - val_loss: 0.5669 - val_accuracy: 0.6652
Epoch 4/8
51/51 [==============================] - 127s 2s/step - loss: 0.5686 - accuracy: 0.6726 - val_loss: 0.5659 - val_accuracy: 0.6652
Epoch 5/8
51/51 [==============================] - 141s 3s/step - loss: 0.5692 - accuracy: 0.6769 - val_loss: 0.5596 - val_accuracy: 0.6652
Epoch 6/8
51/51 [==============================] - 133s 3s/step - loss: 0.5696 - accuracy: 0.6755 - val_loss: 0.5666 - val_accuracy: 0.6652
Epoch 7/8
51/51 [==============================] - 129s 3s/step - loss: 0.5683 - accuracy: 0.6735 - val_loss: 0.5693 - val_accuracy: 0.6652
Epoch 8/8
51/51 [============

In [15]:
# Evaluate the LSTM model
loss, accuracy = model.evaluate(X_test.toarray(), y_test, batch_size=1)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

1631/1631 [==============================] - 48s 29ms/step - loss: 0.5674 - accuracy: 0.6652
Test Loss: 0.5674421191215515
Test Accuracy: 0.6652360558509827


In [16]:
y_train = y_train.reshape(-1,1)
y_train

array([[1],
       [1],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [17]:
labels_pred = model.predict(X_test.toarray())
labels_pred = np.round(labels_pred.flatten())
print(classification_report(y_test, labels_pred))

51/51 [==============================] - 10s 187ms/step
              precision    recall  f1-score   support

           0       0.56      0.77      0.65       649
           1       0.80      0.60      0.68       982

    accuracy                           0.67      1631
   macro avg       0.68      0.68      0.66      1631
weighted avg       0.70      0.67      0.67      1631



In [18]:
def predict(text):
    TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            tokens.append(stemmer.stem(token))
    text = " ".join(tokens)
    text = np.array([text])
    text = vectorizer.transform(text)
    return "POSITIVE" if model.predict(text.toarray()) > 0.5 else "NEGATIVE"

In [19]:
predict("depression is real, life is very hard, i hate everything, suicide is only way")

1/1 [==============================] - 0s 40ms/step


'POSITIVE'

In [20]:
predict("i enjoyed the vacations, i loved listening to songs when i was in vacation")

1/1 [==============================] - 0s 39ms/step


'NEGATIVE'

CNN

In [21]:
# Build the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(tfidf_features.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
X_train_reshaped = X_train.toarray().reshape(X_train.shape[0], tfidf_features.shape[1], 1)
X_test_reshaped = X_test.toarray().reshape(X_test.shape[0], tfidf_features.shape[1], 1)

In [23]:
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=1, validation_data=(X_test_reshaped, y_test))


Epoch 1/10
6524/6524 [==============================] - 18s 3ms/step - loss: 0.3519 - accuracy: 0.8351 - val_loss: 0.1523 - val_accuracy: 0.9430
Epoch 2/10
6524/6524 [==============================] - 17s 3ms/step - loss: 0.1491 - accuracy: 0.9460 - val_loss: 0.1456 - val_accuracy: 0.9418
Epoch 3/10
6524/6524 [==============================] - 17s 3ms/step - loss: 0.1368 - accuracy: 0.9500 - val_loss: 0.1513 - val_accuracy: 0.9467
Epoch 4/10
6524/6524 [==============================] - 16s 3ms/step - loss: 0.1322 - accuracy: 0.9491 - val_loss: 0.1205 - val_accuracy: 0.9546
Epoch 5/10
6524/6524 [==============================] - 16s 2ms/step - loss: 0.1220 - accuracy: 0.9552 - val_loss: 0.1152 - val_accuracy: 0.9552
Epoch 6/10
6524/6524 [==============================] - 16s 3ms/step - loss: 0.1184 - accuracy: 0.9536 - val_loss: 0.1386 - val_accuracy: 0.9454
Epoch 7/10
6524/6524 [==============================] - 16s 2ms/step - loss: 0.1181 - accuracy: 0.9546 - val_loss: 0.1358 - val_ac

In [24]:
labels_pred = model.predict(X_test_reshaped)
labels_pred = np.round(labels_pred.flatten())
print(classification_report(y_test, labels_pred))

51/51 [==============================] - 0s 8ms/step
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       649
           1       0.98      0.94      0.96       982

    accuracy                           0.95      1631
   macro avg       0.95      0.95      0.95      1631
weighted avg       0.95      0.95      0.95      1631



**Hybrid**


In [35]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(tfidf_features.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [36]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 998, 64)           256       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 499, 64)          0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 497, 128)          24704     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 248, 128)         0         
 1D)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_3 (Dense)             (None, 1)                

In [38]:
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=1, validation_data=(X_test_reshaped, y_test))


Epoch 1/10
6524/6524 [==============================] - 277s 42ms/step - loss: 0.6878 - accuracy: 0.5587 - val_loss: 0.6703 - val_accuracy: 0.6058
Epoch 2/10
6524/6524 [==============================] - 270s 41ms/step - loss: 0.6856 - accuracy: 0.5647 - val_loss: 0.6715 - val_accuracy: 0.6101
Epoch 3/10
6524/6524 [==============================] - 270s 41ms/step - loss: 0.6824 - accuracy: 0.5737 - val_loss: 0.6724 - val_accuracy: 0.6162
Epoch 4/10
6524/6524 [==============================] - 270s 41ms/step - loss: 0.6756 - accuracy: 0.5799 - val_loss: 0.6685 - val_accuracy: 0.6174
Epoch 5/10
6524/6524 [==============================] - 270s 41ms/step - loss: 0.6658 - accuracy: 0.5826 - val_loss: 0.6598 - val_accuracy: 0.6205
Epoch 6/10
6524/6524 [==============================] - 270s 41ms/step - loss: 0.6699 - accuracy: 0.5823 - val_loss: 0.6699 - val_accuracy: 0.6033
Epoch 7/10
6524/6524 [==============================] - 273s 42ms/step - loss: 0.6670 - accuracy: 0.5972 - val_loss: 0

In [40]:
labels_pred = model.predict(X_test_reshaped)
labels_pred = np.round(labels_pred.flatten())
print(classification_report(y_test, labels_pred))

51/51 [==============================] - 3s 53ms/step
              precision    recall  f1-score   support

           0       0.88      0.92      0.90       649
           1       0.95      0.92      0.93       982

    accuracy                           0.92      1631
   macro avg       0.91      0.92      0.92      1631
weighted avg       0.92      0.92      0.92      1631

